In [ ]:
%load_ext autoreload
%autoreload 2

# Elastic API prototype
## Steps 
1. Generate docs
2. Push docs to elastic
3. Inference agent
4. Search



# 1. Generate docs
Loads the api specs and guides to create a ndjson elastic can ingest.

Sources
- all guides from https://github.com/elastic/elasticsearch/tree/main/docs/reference
- api spec from https://github.com/elastic/elasticsearch/tree/main/rest-api-spec

In [ ]:
# TODO: Add generation using openai to improve search

def generate_tags(text):
    raise Exception("")

def generate_target_outcomes(text):
    raise Exception("")

def generate_summary(text):
    raise Exception("")

def generate_questions_this_answers(text):
    raise Exception("")

### Docs formatters

In [ ]:
from datetime import datetime
import json
import re

from format_docs import (
    format_elastic_guides,
    format_elastic_api_specs,
)


### Turn data to documents

In [ ]:
from documents import docify


docify("guides")
docify("api-spec")


In [ ]:
from utils.file import generate_ndjson_from_docs_folder

# TODO: HERE

generate_ndjson_from_docs_folder('_docs/api-spec', '_docs/ndjson/api-spec.ndjson')
generate_ndjson_from_docs_folder('_docs/guides', '_docs/ndjson/guides.ndjson')

# Push files to elastic


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
from elasticsearch import Elasticsearch
import os
from dotenv import load_dotenv

load_dotenv()

ES_LOCAL_HOST = os.getenv("ES_LOCAL_HOST")
ES_LOCAL_API_KEY = os.getenv("ES_LOCAL_API_KEY")


client = Elasticsearch(
    ES_LOCAL_HOST,
    api_key=ES_LOCAL_API_KEY,
)
client.info()

In [ ]:
# client.indices.delete(index=index_name, ignore_unavailable=True)


In [ ]:
index_name = "api-spec"


mappings = {
    "properties": {
        "api_name_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        },
        "paths_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        },
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        },
        "doc_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        },
    }
}

client.indices.create(index=index_name, mappings=mappings)

In [ ]:
import json

def process_ndjson(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)


operations = []
for spec in process_ndjson("docs/api-spec.ndjson"):
    print(spec["meta"]["api_name"])
    
    
    operations.append({"index": {"_index": index_name}})

    spec["api_name_vector"] = model.encode(spec["meta"]["api_name"]).tolist()
    break 

    
    spec["paths_vector"] = model.encode(spec["meta"]["paths"]).tolist()
    spec["title_vector"] = model.encode(spec["meta"]["title"]).tolist()
    spec["doc_vector"] = model.encode(spec["doc"]).tolist()

    operations.append(spec)

# client.bulk(index=index_name, operations=operations, refresh=True)

# Training


https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb
https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/03-ELSER.ipynb
https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/integrations/cohere/inference-cohere.ipynb
https://www.elastic.co/training/elasticsearch-engineer
https://www.elastic.co/training/elastic-certified-engineer-exam
https://github.com/mr1716/Elastic-Certified-Engineer-Exam-8.1
https://github.com/LisaHJung/Beginners-guide-to-creating-a-full-stack-JavaScript-app-with-Elasticsearch/tree/main
https://www.elastic.co/guide/en/elasticsearch/reference/current/security-api.html




Data Management

Define an index that satisfies a given set of requirements
Define and use an index template for a given pattern that satisfies a given set of requirements
Define and use a dynamic template that satisfies a given set of requirements
Define an Index Lifecycle Management policy for a time-series index
Define an index template that creates a new data stream
Searching Data

Write and execute a search query for terms and/or phrases in one or more fields of an index
Write and execute a search query that is a Boolean combination of multiple queries and filters
Write an asynchronous search
Write and execute metric and bucket aggregations
Write and execute aggregations that contain sub-aggregations
Write and execute a query that searches across multiple clusters
Write and execute a search that utilizes a runtime field
Developing Search Applications

Highlight the search terms in the response of a query
Sort the results of a query by a given set of requirements
Implement pagination of the results of a search query
Define and use index aliases
Define and use a search template
Data Processing

Define a mapping that satisfies a given set of requirements
Define and use a custom analyzer that satisfies a given set of requirements
Define and use multi-fields with different data types and/or analyzers
Use the Reindex API and Update By Query API to reindex and/or update documents
Define and use an ingest pipeline that satisfies a given set of requirements, including the use of Painless to modify documents
 Define runtime fields to retrieve custom values using Painless scripting
Cluster Management

Diagnose shard issues and repair a cluster's health
Backup and restore a cluster and/or specific indices
Configure a snapshot to be searchable
Configure a cluster for cross-cluster search
Implement cross-cluster replication